In [2]:
# ===================================================================
#                             2ESR
#          558385 - Alexia Ramalho Izidio Dos Santos
#          554746 - Beatriz Vieira de Novais
#          559008 - Hellen Aparecida Moura Silva
#          557397 - Lorenzo Adinolfi Acquesta
#          558859 - Wendell Dos Santos Silva
# ===================================================================

In [4]:
import sys

# --- Configuração do Problema ---

# Para "melhorar a visibilidade", assumimos que a empresa Visalytica agora
# tem uma PREVISÃO de demanda para os próximos T dias.
# Esta previsão pode ser obtida da análise de dados
# históricos (coletados na Sprint 3).

# T = 12 dias de horizonte de planejamento
demands = [20, 10, 0, 30, 15, 5, 0, 10, 40, 5, 15, 25]
T = len(demands)

# Custos
K = 100  # Custo fixo por PEDIDO (frete, administrativo)
h = 2    # Custo de ESTOQUE (por unidade, por dia)

# Ajuste do limite de recursão para a versão recursiva pura
sys.setrecursionlimit(2000)

In [5]:
print("="*50)
print(f"Iniciando Simulação de Otimização de Estoque")
print(f"Horizonte de Planejamento: {T} dias")
print(f"Custo Fixo por Pedido (K): R$ {K}")
print(f"Custo de Estoque (h): R$ {h} por unidade/dia")
print(f"Demandas Previstas: {demands}")
print("="*50)

Iniciando Simulação de Otimização de Estoque
Horizonte de Planejamento: 12 dias
Custo Fixo por Pedido (K): R$ 100
Custo de Estoque (h): R$ 2 por unidade/dia
Demandas Previstas: [20, 10, 0, 30, 15, 5, 0, 10, 40, 5, 15, 25]


In [6]:
# --- 2. Versão Recursiva Pura (Top-Down) ---

def calculate_holding_cost(t, j, demands, h):
    """Calcula o custo de estoque de um pedido feito em 't' para cobrir até 'j'"""
    cost = 0
    # O item do dia 't' não tem custo de estoque (é consumido).
    # O item do dia 't+1' fica 1 dia em estoque.
    # O item do dia 't+2' fica 2 dias em estoque.
    for i in range(t + 1, j + 1):
        cost += h * (i - t) * demands[i]
    return cost

def solve_recursive(t):
    # Caso Base: Se estamos no dia T ou além, o custo é 0.
    if t >= T:
        return 0

    min_cost = float('inf')

    # Decisão: Tentar todas as possibilidades de 'j' (dia final de cobertura)
    for j in range(t, T):
        # 1. Custo fixo do pedido
        order_cost = K

        # 2. Custo de estoque para este pedido
        holding_cost = calculate_holding_cost(t, j, demands, h)

        # 3. Custo futuro (recursão para o próximo período após 'j')
        future_cost = solve_recursive(t = j + 1)

        # Custo total para ESTA decisão (t, j)
        current_cost = order_cost + holding_cost + future_cost

        if current_cost < min_cost:
            min_cost = current_cost

    return min_cost

In [7]:
# --- 3. Versão com Memorização (Top-Down DP) ---

# Dicionário 'memo' para armazenar resultados já calculados
memo = {}

def solve_memo(t):
    # Caso Base
    if t >= T:
        return 0

    # Se já calculamos C(t), retorna o valor salvo
    if t in memo:
        return memo[t]

    min_cost = float('inf')

    # Decisão: Tentar todas as possibilidades de 'j'
    for j in range(t, T):
        order_cost = K
        holding_cost = calculate_holding_cost(t, j, demands, h)

        # Custo futuro (usando a versão com memorização)
        future_cost = solve_memo(t = j + 1)

        current_cost = order_cost + holding_cost + future_cost

        if current_cost < min_cost:
            min_cost = current_cost

    # Armazena o resultado de C(t) antes de retornar
    memo[t] = min_cost
    return min_cost

In [8]:
# --- 4. Versão Iterativa (Bottom-Up DP) ---

def solve_bottom_up():
    # dp[t] = custo mínimo para os dias t...T-1
    # Precisamos de T+1 posições, pois dp[T] é o caso base.
    dp = [0] * (T + 1)

    # policy[t] = armazena qual 'j' foi a decisão ótima para o dia 't'
    policy = [0] * T

    # Preenchemos a tabela dp DE TRÁS PARA FRENTE
    # Começamos em T-1 (último dia) e vamos até 0 (primeiro dia)
    for t in range(T - 1, -1, -1):

        min_cost = float('inf')
        best_j = t # A melhor decisão 'j' encontrada para este 't'

        # Decisão: Tentar todas as possibilidades de 'j' (de t até T-1)
        for j in range(t, T):
            order_cost = K
            holding_cost = calculate_holding_cost(t, j, demands, h)

            # Custo futuro vem da tabela 'dp' que já foi preenchida
            future_cost = dp[j + 1]

            current_cost = order_cost + holding_cost + future_cost

            if current_cost < min_cost:
                min_cost = current_cost
                best_j = j

        # Armazena o custo mínimo para o dia 't'
        dp[t] = min_cost
        # Armazena a política (decisão) ótima para o dia 't'
        policy[t] = best_j

    # O resultado final é o custo para o dia 0
    return dp[0], policy

In [9]:
# --- 5. Função para Reconstruir a Política de Pedidos ---

def print_policy(policy, demands):
    print("\n--- Política de Pedidos Otimizada ---")
    t = 0
    while t < T:
        j = policy[t]
        quantity = sum(demands[i] for i in range(t, j + 1))

        if quantity > 0:
            print(f"• Dia {t+1}: Fazer pedido de {quantity} unidades.")
            print(f"  (Este pedido cobre a demanda dos dias {t+1} a {j+1})")
        else:
            print(f"• Dia {t+1}: Nenhum pedido (demanda é 0 até dia {j+1}).")

        # O próximo dia para tomar uma decisão é j+1
        t = j + 1
    print("--------------------------------------")

In [10]:
# --- 6. Execução e Verificação ---

print("\n--- Executando Versões da PD ---")

# (A versão recursiva pura é muito lenta, pode ser comentada se T for grande)
# print("Calculando (Recursivo)...")
# cost_rec = solve_recursive(0)
# print(f"Custo Mínimo (Recursivo): R$ {cost_rec}")

print("Calculando (Memorização)...")
cost_memo = solve_memo(0)
print(f"Custo Mínimo (Memorização): R$ {cost_memo}")

print("Calculando (Bottom-Up)...")
cost_bu, policy = solve_bottom_up()
print(f"Custo Mínimo (Bottom-Up): R$ {cost_bu}")

# Verificação
# assert(cost_rec == cost_memo == cost_bu)
assert(cost_memo == cost_bu)

print("\n[VERIFICAÇÃO] Todas as versões de PD produzem o mesmo resultado.")

# Exibe o plano de ação
print_policy(policy, demands)


--- Executando Versões da PD ---
Calculando (Memorização)...
Custo Mínimo (Memorização): R$ 610
Calculando (Bottom-Up)...
Custo Mínimo (Bottom-Up): R$ 610

[VERIFICAÇÃO] Todas as versões de PD produzem o mesmo resultado.

--- Política de Pedidos Otimizada ---
• Dia 1: Fazer pedido de 30 unidades.
  (Este pedido cobre a demanda dos dias 1 a 3)
• Dia 4: Fazer pedido de 60 unidades.
  (Este pedido cobre a demanda dos dias 4 a 8)
• Dia 9: Fazer pedido de 45 unidades.
  (Este pedido cobre a demanda dos dias 9 a 10)
• Dia 11: Fazer pedido de 40 unidades.
  (Este pedido cobre a demanda dos dias 11 a 12)
--------------------------------------
